# *Insight*-HXMT 相位分解谱处理示例
## ——[概览](#概览)、[数据预处理](#数据预处理)、[计时分析](#计时分析)、[能谱分析](#能谱分析)

庹攸隶(tuoyl@ihep.ac.cn)
##### 最终结果：使用慧眼一次 Crab 的观测数据，产生 Crab 脉冲星的相位分解谱

## 概览
### 准备工作
该 Jupyter 文本使用 Python3 环境，若想执行以下所有命令，需要如下操作：
* 安装并初始化 HXMTDAS 环境（例如能在终端中运行```hepical```命令）
* 使用 Python3.* 版本，并安装有 astropy, numpy, matplotlib 模块
    （若使用 conda 环境，请在 environment.yml 文件所在路径执行 ``` conda env create -f environment.yml ``` 安装名为 hxmt_analysis 的环境，然后执行 ```conda activate hxmt_analysis```
    <div class="alert alert-block alert-info">
<b>NOTES
    :</b> 该介绍中涉及到的命令，你可以在 Jupyter 中使用 Shift+Enter 逐条执行。你同时完全可以在终端中执行所有命令。
</div>
    
### 目标
* **数据预处理**：使用慧眼用户处理软件（HXMTDAS v2.01）产生用于分析的数据产品
* **计时分析**：使用 Crab 的星历（ephemeris）产生 Crab 的脉冲轮廓
* **能谱分析**：对轮廓分成多个相位区间，得到各个区间的能谱及背景能谱

## 数据预处理
数据预处理针对三个载荷高能、中能和低等载荷，分别是：
* [高能载荷(HE)数据处理](#高能载荷(HE)数据处理)
* [中能载荷(ME)数据处理](#中能载荷(ME)数据处理)
* [低能载荷(LE)数据处理](#低能载荷(LE)数据处理)

### 高能载荷(HE)数据处理
### hepical

In [4]:
!hepical evtfile=./data/HE/HXMT_P010129900101_HE-Evt_FFFFFF_V1_L1P.FITS outfile=./data/HE/he_pi.fits clobber=yes

hepical : ##############################################
hepical : HXMT HE task, hepical is running
hepical : PILParSet Warning: parameter 'clobber' set to yes!
hepical : PILParSet Warning: the file ./data/HE/he_pi.fits will be overwritten!
hepical : HxmtCaldb Error: There are not files that satisfied the selection criteria!
hepical : HxmtCaldb Error: 'Telescope' HXMT 'instrument' HE 'detname' NONE 'filter' NONE 'codename' CHAN2PI_0' 
hepical : hepical: Error: Unable to get 'gain (codename:CHAN2PI_0)' file named ''!
hepical : HXMT HE task, hepical is running unsuccessfully!
hepical : ##############################################


<div class="alert alert-block alert-warning">
<b>NOTES:</b> 若出现报错 hepical : hepical: Error: Unable to get 'gain (codename:CHAN2PI_0)' file named ''! 则根据 CALDB 中 gainfile 的路径手动指定，执行如下命令。
</div>

In [7]:
!hepical evtfile=./data/HE/HXMT_P010129900101_HE-Evt_FFFFFF_V1_L1P.FITS gainfile=/Users/tuoyouli/Documents/hxmtsoft_newtest/CALDB/data/hxmt/he/bcf/hxmt_he_gain_20171030_v1.fits outfile=./data/HE/he_pi.fits clobber=yes

hepical : ##############################################
hepical : HXMT HE task, hepical is running
[####################################################################################################][100%]
hepical : HXMT HE task, all events from raw evt file is 145630697!
hepical : HXMT HE task, glitch removal events is 636135!
hepical : HXMT HE task, hepical is running successfully!
hepical : ##############################################


运行该命令根据不同的计算机性能，通常会占用你2-3分钟的时间。输出产生一个新的事例文件，命名为 he_pi.fits。

In [8]:
!ls -trl ./data/HE/

total 9658648
-rwx------  1 tuoyouli  staff     1143360 Aug 19 15:31 HXMT_P010129900101_HE-HV_FFFFFF_V1_L1P.FITS
-rwx------  1 tuoyouli  staff      469440 Aug 19 15:31 HXMT_P010129900101_HE-DTime_FFFFFF_V1_L1P.FITS
-rwx------  1 tuoyouli  staff      109440 Aug 19 15:31 HXMT_P010129900101_HE-PM_FFFFFF_V1_L1P.FITS
-rwx------  1 tuoyouli  staff     1673280 Aug 19 15:31 HXMT_P010129900101_HE-TH_FFFFFF_V1_L1P.FITS
-rw-r--r--  1 tuoyouli  staff  2330262720 Aug 19 15:31 HXMT_P010129900101_HE-Evt_FFFFFF_V1_L1P.FITS
-rwx------  1 tuoyouli  staff      149760 Aug 19 15:31 HXMT_P010129900101_HE-InsStat_FFFFFF_V1_L1P.FITS
-rw-r--r--  1 tuoyouli  staff  2610072000 Aug 19 20:18 he_pi.fits


### hegtigen
生成 HE 载荷的好时间文件（GTIs）

In [11]:
!hegtigen hvfile=./data/HE/HXMT_P010129900101_HE-HV_FFFFFF_V1_L1P.FITS tempfile=./data/HE/HXMT_P010129900101_HE-TH_FFFFFF_V1_L1P.FITS ehkfile=./data/AUX/HXMT_P010129900101_HE-EHK_FFFFFF_V1_L1P.FITS outfile=./data/HE/he_gti.fits defaultexpr="NONE" expr="ELV>10&&COR>8&&T_SAA>=300&&TN_SAA>=300&&ANG_DIST<=0.04" clobber=yes

hegtigen : ##############################################
hegtigen : HXMT HE task, hegtigen is running
hegtigen : PILParSet Warning: parameter 'clobber' set to yes!
hegtigen : PILParSet Warning: the file ./data/HE/he_gti.fits will be overwritten!
hegtigen : HXMT HE task, hegtigen is running successfully!
hegtigen : ##############################################


我们推荐的对于涉及到能谱分析的好时间判选条件为```ELV>10&&COR>8&&T_SAA>=300&&TN_SAA>=300&&ANG_DIST<=0.04```。
同样的，你可以查看输出文件，我们这一步产生了一个命名为 he_gti.fits 的 FITS 文件。

In [12]:
!ls -trl ./data/HE/

total 9658688
-rwx------  1 tuoyouli  staff     1143360 Aug 19 15:31 HXMT_P010129900101_HE-HV_FFFFFF_V1_L1P.FITS
-rwx------  1 tuoyouli  staff      469440 Aug 19 15:31 HXMT_P010129900101_HE-DTime_FFFFFF_V1_L1P.FITS
-rwx------  1 tuoyouli  staff      109440 Aug 19 15:31 HXMT_P010129900101_HE-PM_FFFFFF_V1_L1P.FITS
-rwx------  1 tuoyouli  staff     1673280 Aug 19 15:31 HXMT_P010129900101_HE-TH_FFFFFF_V1_L1P.FITS
-rw-r--r--  1 tuoyouli  staff  2330262720 Aug 19 15:31 HXMT_P010129900101_HE-Evt_FFFFFF_V1_L1P.FITS
-rwx------  1 tuoyouli  staff      149760 Aug 19 15:31 HXMT_P010129900101_HE-InsStat_FFFFFF_V1_L1P.FITS
-rw-r--r--  1 tuoyouli  staff  2610072000 Aug 19 20:18 he_pi.fits
-rw-r--r--  1 tuoyouli  staff       17280 Aug 19 20:19 he_gti.fits


### hescreen
根据 GTIs 选择出符合要求的好事例

In [1]:
!hescreen evtfile=./data/HE/he_pi.fits gtifile=./data/HE/he_gti.fits outfile=./data/HE/he_screen.fits userdetid=0-17 minPI=0 maxPI=255 clobber=yes

hescreen : ##############################################
hescreen : HXMT HE task, hescreen is running
hescreen : PILParSet Warning: parameter 'clobber' set to yes!
hescreen : PILParSet Warning: the file ./data/HE/he_screen.fits will be overwritten!
hescreen : hescreen: User detector selection:
hescreen : HxmtBadDetector Info: there is no bad detector!
[####################################################################################################][100%]
hescreen : HEScreen task: 
hescreen : Detector ID '0' has event number :237400
hescreen : Detector ID '1' has event number :244273
hescreen : Detector ID '2' has event number :263201
hescreen : Detector ID '3' has event number :247841
hescreen : Detector ID '4' has event number :236576
hescreen : Detector ID '5' has event number :242721
hescreen : Detector ID '6' has event number :252674
hescreen : Detector ID '7' has event number :237050
hescreen : Detector ID '8' has event number :261260
hescreen : Detector ID '9' has event numb

运行 hescreen 命令大约会占用你1分钟时间，输出文件为 he_screen.fits。我们这里选择了探测器编号0-17，即全选，选择了能道0-255，也是全选。如果你选择产生不同能段的光子，则可以修改 minPI 和 maxPI 的值。

### hespecgen 
生成能谱文件（spectra）

In [1]:
!hespecgen evtfile=./data/HE/he_screen.fits outfile=./data/HE/he_spec deadfile=./data/HE/HXMT_P010129900101_HE-DTime_FFFFFF_V1_L1P.FITS userdetid="0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17" starttime=0 stoptime=0 minPI=0 maxPI=255 clobber=yes

hespecgen : HXMT HE task, hespecgen is running
hespecgen : hespecgen: User detector selection:
[####################################################################################################][100%]
hespecgen : HXMT HE task, hespecgen is running successfully!
hespecgen : ##############################################


我们在这一步生成了 HE 的18个探测器的能谱，通过 userdetid 参数选择探测器。使用分号(;)将各个探测器编号做分割，使我们产生了18个能谱，而不是18个探测器的总能谱。可以查看一下我们产生的结果，是18个以```he_spec```为前缀的```.pha```文件。能谱的结果可以在[能谱分析](#能谱分析)一节查看，这里暂不做展开。

In [6]:
!ls -tr ./data/HE/

HXMT_P010129900101_HE-HV_FFFFFF_V1_L1P.FITS	  he_spec_g4_4.pha
HXMT_P010129900101_HE-DTime_FFFFFF_V1_L1P.FITS	  he_spec_g3_3.pha
he_pi.fits					  he_spec_g2_2.pha
he_gti.fits					  he_spec_g17_17.pha
HXMT_P010129900101_HE-PM_FFFFFF_V1_L1P.FITS	  he_spec_g16_16.pha
he_screen.fits					  he_spec_g15_15.pha
HXMT_P010129900101_HE-TH_FFFFFF_V1_L1P.FITS	  he_spec_g14_14.pha
HXMT_P010129900101_HE-InsStat_FFFFFF_V1_L1P.FITS  he_spec_g13_13.pha
HXMT_P010129900101_HE-Evt_FFFFFF_V1_L1P.FITS	  he_spec_g12_12.pha
he_spec_g9_9.pha				  he_spec_g1_1.pha
he_spec_g8_8.pha				  he_spec_g11_11.pha
he_spec_g7_7.pha				  he_spec_g10_10.pha
he_spec_g6_6.pha				  he_spec_g0_0.pha
he_spec_g5_5.pha


<div class="alert alert-block alert-info">
<b>NOTES:</b> 我们之所以要产生18个能谱而不是一个总谱，是由于各个探测器的响应矩阵不一样，我们在之后会根据各个探测器的能谱产生其各自的响应（盲探测器不产生）。最后可以使用脚本 hhe_spec2pi 将能谱和响应矩阵合并。
</div>

In [1]:
!ls ./data/HE/he_spec*.pha | sort -V > ./data/HE/he_spec.txt
!cat ./data/HE/he_spec.txt

./data/HE/he_spec_g0_0.pha
./data/HE/he_spec_g1_1.pha
./data/HE/he_spec_g2_2.pha
./data/HE/he_spec_g3_3.pha
./data/HE/he_spec_g4_4.pha
./data/HE/he_spec_g5_5.pha
./data/HE/he_spec_g6_6.pha
./data/HE/he_spec_g7_7.pha
./data/HE/he_spec_g8_8.pha
./data/HE/he_spec_g9_9.pha
./data/HE/he_spec_g10_10.pha
./data/HE/he_spec_g11_11.pha
./data/HE/he_spec_g12_12.pha
./data/HE/he_spec_g13_13.pha
./data/HE/he_spec_g14_14.pha
./data/HE/he_spec_g15_15.pha
./data/HE/he_spec_g16_16.pha
./data/HE/he_spec_g17_17.pha


### herspgen
产生除盲探测器外，所有探测器各自的响应矩阵

In [1]:
phalist = open("./data/HE/he_spec.txt")
for phafile,i in zip(phalist,range(18)):
    if i == 16:continue # 16号盲探测器不产生响应矩阵
    herspgen_text = "herspgen phafile=%s outfile=./data/HE/he_rsp_g%s.fits attfile=%s ra=-1 dec=-91 clobber=yes"%(phafile[0:-1], str(i), "./data/ACS/HXMT_P010129900101_Att_FFFFFF_V1_L1P.FITS")
    !{herspgen_text}
phalist.close()

herspgen : HXMT HE task, herspgen is running
herspgen : HERspGen: Attitude and Alignment correction of detector '0'  is 0.985351 !
herspgen : HXMT HE task, herspgen is running successfully!
herspgen : ##############################################
herspgen : HXMT HE task, herspgen is running
herspgen : HERspGen: Attitude and Alignment correction of detector '1'  is 0.980126 !
herspgen : HXMT HE task, herspgen is running successfully!
herspgen : ##############################################
herspgen : HXMT HE task, herspgen is running
herspgen : HERspGen: Attitude and Alignment correction of detector '2'  is 0.995052 !
herspgen : HXMT HE task, herspgen is running successfully!
herspgen : ##############################################
herspgen : HXMT HE task, herspgen is running
herspgen : HERspGen: Attitude and Alignment correction of detector '3'  is 0.980126 !
herspgen : HXMT HE task, herspgen is running successfully!
herspgen : ##############################################
herspgen

我们产生了17个探测器的响应矩阵。可以查看产生的结果

In [4]:
!ls -tr ./data/HE//he_rsp*

./data/HE//he_rsp_g0.fits  ./data/HE//he_rsp_g9.fits
./data/HE//he_rsp_g1.fits  ./data/HE//he_rsp_g10.fits
./data/HE//he_rsp_g2.fits  ./data/HE//he_rsp_g11.fits
./data/HE//he_rsp_g3.fits  ./data/HE//he_rsp_g12.fits
./data/HE//he_rsp_g4.fits  ./data/HE//he_rsp_g13.fits
./data/HE//he_rsp_g5.fits  ./data/HE//he_rsp_g14.fits
./data/HE//he_rsp_g6.fits  ./data/HE//he_rsp_g15.fits
./data/HE//he_rsp_g7.fits  ./data/HE//he_rsp_g17.fits
./data/HE//he_rsp_g8.fits


### 中能载荷(ME)数据处理

### mepical
ME 探测器的 Pulse Invariance CALibration。
输入的两个文件，```evtfile=``` 输入的是原始的事例文件（文件名的关键字是```ME-Evt```），```tempfile``` 是温度文件（文件名的关键字是```ME-TH```）

In [1]:
!mepical evtfile=./data/ME/HXMT_P010129900101_ME-Evt_FFFFFF_V1_L1P.FITS tempfile=./data/ME/HXMT_P010129900101_ME-TH_FFFFFF_V1_L1P.FITS outfile=./data/ME/me_pi.fits clobber=yes

mepical : ##############################################
mepical : HXMT ME task, mepical is running
[####################################################################################################][100%]
mepical : HXMT ME task, mepical is running successfully!
mepical : ##############################################


我们可以查看新产生的文件

In [4]:
!ls -l ./data/ME/me_pi.fits

-rw-r--r--  1 tuoyouli  staff  2267202240 Aug 23 21:29 ./data/ME/me_pi.fits


### megrade
对 ME 的事例进行分级，并挑选“单分裂”事例。同时可以产生 ME 的死时间文件。```evtfile``` 是输入的文件名，该文件是上一步 ```mepical``` 输出产生的文件。```outfile``` 和 ```deadfile``` 分别为输出的完成事例挑选的文件名和输出的死时间文件的文件名。

In [5]:
!megrade evtfile=./data/ME/me_pi.fits outfile=./data/ME/me_grade.fits deadfile=./data/ME/me_dtime.fits clobber=yes

megrade : ##############################################
megrade : HXMT ME task, megrade is running
[####################################################################################################][100%]
megrade : HXMT ME task, megrade is running successfully!
megrade : ##############################################


### megtigen
产生 ME 探测器对应的好时间文件。好时间的判选条件和 HE 的选择条件相同，为```ELV>10&&COR>8&&T_SAA>=300&&TN_SAA>=300&&ANG_DIST<=0.04```

In [1]:
!megtigen tempfile=./data/ME/HXMT_P010129900101_ME-TH_FFFFFF_V1_L1P.FITS ehkfile=./data/AUX/HXMT_P010129900101_HE-EHK_FFFFFF_V1_L1P.FITS outfile=./data/ME/me_gti.fits defaultexpr="NONE" expr="ELV>10&&COR>8&&T_SAA>=300&&TN_SAA>=300&&ANG_DIST<=0.04" clobber=yes

megtigen : ##############################################
megtigen : HXMT ME task, megtigen is running
megtigen : PILParSet Warning: parameter 'clobber' set to yes!
megtigen : PILParSet Warning: the file ./data/ME/me_gti.fits will be overwritten!
megtigen : HXMT ME task, megtigen is running successfully!
megtigen : ##############################################


### megti

In [2]:
!python ./hxmt_scripts/megti.py -h

('----------', 'import end', '----------')
*********************************************************
*********************************************************
*********************************************************
************ PRINT: megti -h for usage   *************
*********************************************************
Method 1: megti megrade.fits oldgti.fits newgti.fits
Method 2: Using interactive method in prompt.


In [3]:
!python ./hxmt_scripts/megti.py ./data/ME/me_grade.fits ./data/ME/me_gti.fits ./data/ME/me_gti_new.fits

('----------', 'import end', '----------')
*********************************************************
*********************************************************
*********************************************************
************ PRINT: megti -h for usage   *************
*********************************************************
(array([1.78432393e+08, 1.78433313e+08, 1.78433360e+08, 1.78433467e+08,
       1.78438215e+08, 1.78439098e+08, 1.78439107e+08, 1.78439111e+08,
       1.78439115e+08, 1.78444099e+08]), array([1.78433299e+08, 1.78433340e+08, 1.78433419e+08, 1.78433780e+08,
       1.78439097e+08, 1.78439106e+08, 1.78439109e+08, 1.78439113e+08,
       1.78439462e+08, 1.78444844e+08]))
chooseing the gradedata in the input gtifile
(29385504, 29385504)
('median==', -31.215000000000032)
('loop1', 'median=', -31.215000000000032, 'Dmedian=', 1000031.215)
('median==', -31.215000000000032)
('loop2', 'median=', -31.215000000000032, 'Dmedian=', 0.0)
[904.  58. 312. 881. 346. 521. 212.]
done


In [4]:
!ls -1t ./data/ME/me_gti*

./data/ME/me_gti_new.fits
./data/ME/me_gti.fits


### mescreen

In [5]:
!python ./hxmt_scripts/hprint_detid.py 

*********** Detector ID catalogue ***********
--------------------------
--------------------------

HE detector ID for SMALL FOV (5.7x1.1):
0, 1, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 17

HE BLIND detector ID:
16

HE SMALL FOV information (5.7x1.1):
FOV   0: 1, 3, 8, 11, 12
FOV -60: 0, 4, 7,  9, 14, 17
FOV +60: 2, 5, 6, 10, 13, 15

HE LARGE FOV information (5.7x5.7):
2, 9
--------------------------
--------------------------

ME ASIC ID for SMALL FOV (1x4):
0-5, 7, 12-23, 25, 30-41, 43, 48-53

ME BLIND ASIC ID:
10, 28, 46

ME ASIC ID with calibration source:
6, 11, 24, 29, 42, 47

ME SMALL FOV information (1x4):
FOV 0:   0-5,  7, 12-17
FOV 1: 18-23, 25, 30-35
FOV 2: 36-41, 43, 48-53

ME LARGE FOV information (4x4):
8, 9, 26, 27, 44, 45
--------------------------
--------------------------

LE detector ID for SMALL FOV (1.6x6):
0,2-4,6-10,12,14,20,22-26,28,30,32,34-36,38-42,44,46,52,54-58,60-62,64,66-68,70-74,76,78,84,86,88-90,92-94

LE BLIND detector ID for SMALL FOV:
13, 45, 77



In [6]:
!mescreen evtfile=./data/ME/me_grade.fits gtifile=./data/ME/me_gti.fits outfile=./data/ME/me_screen.fits userdetid="0-5,7,12-23,25,30-41,43,48-53;10,28,46" clobber=yes 

mescreen : ##############################################
mescreen : HXMT ME task, mescreen is running
mescreen : PILParSet Warning: parameter 'clobber' set to yes!
mescreen : PILParSet Warning: the file ./data/ME/me_screen.fits will be overwritten!
mescreen : mescreen: User detector selection:
mescreen : Bad (Bad,hot,flickening) detector List:
mescreen :        ID:  1
mescreen :        ID:  6
mescreen :        ID:  8
mescreen :        ID:  9
mescreen :        ID:  14
mescreen :        ID:  15
mescreen :        ID:  16
mescreen :        ID:  17
mescreen :        ID:  18
mescreen :        ID:  19
mescreen :        ID:  20
mescreen :        ID:  28
mescreen :        ID:  30
mescreen :        ID:  31
mescreen :        ID:  32
mescreen :        ID:  33
mescreen :        ID:  48
mescreen :        ID:  96
mescreen :        ID:  97
mescreen :        ID:  161
mescreen :        ID:  186
mescreen :        ID:  187
mescreen :        ID:  188
mescreen :        ID:  189
mescreen :        ID:  190
m

[####################################################################################################][100%]
mescreen : MEScreen task: 
mescreen : Detector ID '0' has event number :30675
mescreen : Detector ID '1' has event number :47373
mescreen : Detector ID '2' has event number :51318
mescreen : Detector ID '3' has event number :48772
mescreen : Detector ID '4' has event number :52885
mescreen : Detector ID '5' has event number :38759
mescreen : Detector ID '7' has event number :37019
mescreen : Detector ID '10' has event number :20294
mescreen : Detector ID '12' has event number :44241
mescreen : Detector ID '13' has event number :43148
mescreen : Detector ID '14' has event number :43662
mescreen : Detector ID '15' has event number :43319
mescreen : Detector ID '16' has event number :54026
mescreen : Detector ID '17' has event number :48136
mescreen : Detector ID '18' has event number :51227
mescreen : Detector ID '19' has event number :35722
mescreen : Detector ID '20' has event n

### mespecgen

In [7]:
!mespecgen evtfile=./data/ME/me_screen.fits outfile=./data/ME/me_spec deadfile=./data/ME/me_dtime.fits userdetid="0-5,7,12-23,25,30-41,43,48-53" clobber=yes

mespecgen : HXMT ME task, mespecgen is running
mespecgen : mespecgen: User detector selection:
[####################################################################################################][100%]
mespecgen : PILParSet Warning: parameter 'clobber' set to yes!
mespecgen : PILParSet Warning: the file ./data/ME/me_spec_g0_0-53.pha will be overwritten!
mespecgen : HXMT ME task, mespecgen is running successfully!
mespecgen : ##############################################


In [8]:
!ls ./data/ME/me_spec* 

./data/ME/me_spec_g0_0-53.pha


### merspgen

In [9]:
!merspgen phafile=./data/ME/me_spec_g0_0-53.pha outfile=./data/ME/me_rsp.fits attfile=./data/ACS/HXMT_P010129900101_Att_FFFFFF_V1_L1P.FITS ra=-1 dec=-91 clobber=yes

merspgen : HXMT ME task, merspgen is running
merspgen : PILParSet Warning: parameter 'clobber' set to yes!
merspgen : PILParSet Warning: the file ./data/ME/me_rsp.fits will be overwritten!
merspgen : MERspGen: detector id '0' will use common ARF!
merspgen : MERspGen: detector id '0' will use common RMF!
merspgen : MERspGen: Attitude and Alignment correction of detector '0'  is 0.979575 !
merspgen : MERspGen: detector id '1' will use common ARF!
merspgen : MERspGen: detector id '1' will use common RMF!
merspgen : MERspGen: Attitude and Alignment correction of detector '1'  is 0.979575 !
merspgen : MERspGen: detector id '2' will use common ARF!
merspgen : MERspGen: detector id '2' will use common RMF!
merspgen : MERspGen: Attitude and Alignment correction of detector '2'  is 0.979575 !
merspgen : MERspGen: detector id '3' will use common ARF!
merspgen : MERspGen: detector id '3' will use common RMF!
merspgen : MERspGen: Attitude and Alignment correction of detector '3'  is 0.979575 !
mer

In [10]:
!ls ./data/ME/me_rsp.fits

./data/ME/me_rsp.fits


### 低能载荷(LE)数据处理

### lepical
LE 探测器的 Pulse Invariance CALibration

In [2]:
!lepical evtfile=./data/LE/HXMT_P010129900101_LE-Evt_FFFFFF_V1_L1P.FITS tempfile=./data/LE/HXMT_P010129900101_LE-TH_FFFFFF_V1_L1P.FITS outfile=./data/LE/le_pi.fits

lepical : ##############################################
lepical : HXMT LE task, lepical is running
[####################################################################################################][100%]
lepical : HXMT LE task, lepical is running successfully!
lepical : ##############################################


产生了一个命名为 le_pi.fits 的事例文件。可以查看新产生的文件

In [13]:
!ls ./data/LE/le_pi.fits

./data/LE/le_pi.fits


In [14]:
!lerecon evtfile=./data/LE/le_pi.fits outfile=./data/LE/le_recon.fits instatusfile=./data/LE/HXMT_P010129900101_LE-InsStat_FFFFFF_V1_L1P.FITS

lerecon : ##############################################
lerecon : HXMT LE task, lerecon is running
lerecon : PILParSet Warning: parameter 'clobber' set to yes!
lerecon : PILParSet Warning: the file ./data/LE/le_recon.fits will be overwritten!
[####################################################################################################][100%]
lerecon : HXMT LE task, lerecon is running successfully!
lerecon : ##############################################


In [15]:
!legtigen evtfile="" instatusfile=./data/LE/HXMT_P010129900101_LE-InsStat_FFFFFF_V1_L1P.FITS tempfile=./data/LE/HXMT_P010129900101_LE-TH_FFFFFF_V1_L1P.FITS ehkfile=./data/AUX/HXMT_P010129900101_HE-EHK_FFFFFF_V1_L1P.FITS outfile=./data/LE/le_gti.fits defaultexpr="NONE" expr="ELV>10&&COR>8&&DYE_ELV>40&&T_SAA>=300&&TN_SAA>=300&&ANG_DIST<=0.04" clobber=yes

legtigen : ##############################################
legtigen : HXMT LE task, legtigen is running
legtigen : PILParSet Warning: parameter 'clobber' set to yes!
legtigen : PILParSet Warning: the file ./data/LE/le_gti.fits will be overwritten!
legtigen : legtigen: Warning: Unable to get 'EVENT' file! GTI from Noise Events(1000/500) is set to NULL!
legtigen : HXMT LE task, legtigen is running successfully!
legtigen : ##############################################


In [16]:
!python ./hxmt_scripts/legti.py ./data/LE/le_recon.fits ./data/LE/le_gti.fits ./data/LE/le_gti_new.fits

('----------', 'import end', '----------')
*********************************************************
*********************************************************
*********************************************************
************ PRINT: legti -h for usage   *************
*********************************************************
(array([1.78432393e+08, 1.78433360e+08, 1.78433467e+08, 1.78438215e+08,
       1.78439115e+08, 1.78444099e+08]), array([1.78433299e+08, 1.78433419e+08, 1.78433780e+08, 1.78439097e+08,
       1.78439462e+08, 1.78444844e+08]))
chooseing the gradedata in the input gtifile
('factor===', 3.411764705882353)
find the bad data
('cri1=', 8.763341985613465)
('loop1', 'median=', 0.6014705882352924, 'Dmedian=', 999999.3985294118)
('cri1=', 5.573550902859161)
('loop2', 'median=', 0.47156862745097783, 'Dmedian=', 0.1299019607843146)
('cri1=', 5.552705605613449)
('loop3', 'median=', 0.4580882352941176, 'Dmedian=', 0.013480392156860255)
('cri1=', 5.552705605613449)
('loop4', 'm

In [18]:
!ls -1 ./data/LE/le_gti*

./data/LE/le_gti.fits
./data/LE/le_gti_new.fits


In [19]:
!lescreen evtfile=./data/LE/le_recon.fits gtifile=./data/LE/le_gti_new.fits outfile=./data/LE/le_screen.fits userdetid="0,2-4,6-10,12,14,20,22-26,28,30,32,34-36,38-42,44,46,52,54-58,60-62,64,66-68,70-74,76,78,84,86,88-90,92-94,13,45,77,21,53,85" clobber=yes

lescreen : ##############################################
lescreen : HXMT LE task, lescreen is running
lescreen : PILParSet Warning: parameter 'clobber' set to yes!
lescreen : PILParSet Warning: the file ./data/LE/le_screen.fits will be overwritten!
lescreen : lescreen: User detector selection:
lescreen : HxmtBadDetector Info: there is no bad detector!
[####################################################################################################][100%]
lescreen : LEScreen task: 
lescreen : Detector ID '0' has event number :12719
lescreen : Detector ID '2' has event number :12532
lescreen : Detector ID '3' has event number :12544
lescreen : Detector ID '4' has event number :12456
lescreen : Detector ID '6' has event number :12481
lescreen : Detector ID '7' has event number :12497
lescreen : Detector ID '8' has event number :12744
lescreen : Detector ID '9' has event number :12915
lescreen : Detector ID '10' has event number :12257
lescreen : Detector ID '12' has event number :128

In [20]:
!lespecgen evtfile=./data/LE/le_screen.fits outfile=./data/LE/le_spec userdetid="0,2-4,6-10,12,14,20,22-26,28,30,32,34-36,38-42,44,46,52,54-58,60-62,64,66-68,70-74,76,78,84,86,88-90,92-94" eventtype=1 clobber=yes

lespecgen : HXMT LE task, lespecgen is running
[####################################################################################################][100%]
lespecgen : PILParSet Warning: parameter 'clobber' set to yes!
lespecgen : PILParSet Warning: the file ./data/LE/le_spec_g0_0-94.pha will be overwritten!
lespecgen : HXMT LE task, lespecgen is running successfully!
lespecgen : ##############################################


In [21]:
!lerspgen phafile=./data/LE/le_spec_g0_0-94.pha outfile=./data/LE/le_rsp.fits attfile=./data/ACS/HXMT_P010129900101_Att_FFFFFF_V1_L1P.FITS tempfile=./data/LE/HXMT_P010129900101_LE-TH_FFFFFF_V1_L1P.FITS ra=-1 dec=-91 clobber=yes

lerspgen : HXMT LE task, lerspgen is running
lerspgen : PILParSet Warning: parameter 'clobber' set to yes!
lerspgen : PILParSet Warning: the file ./data/LE/le_rsp.fits will be overwritten!
lerspgen : LERspGen: Attitude and Alignment correction of detector '0'  is 0.993464 !
lerspgen : LERspGen: Attitude and Alignment correction of detector '2'  is 0.993464 !
lerspgen : LERspGen: Attitude and Alignment correction of detector '3'  is 0.993464 !
lerspgen : LERspGen: Attitude and Alignment correction of detector '4'  is 0.993464 !
lerspgen : LERspGen: Attitude and Alignment correction of detector '6'  is 0.993464 !
lerspgen : LERspGen: Attitude and Alignment correction of detector '7'  is 0.993464 !
lerspgen : LERspGen: Attitude and Alignment correction of detector '8'  is 0.993464 !
lerspgen : LERspGen: Attitude and Alignment correction of detector '9'  is 0.993464 !
lerspgen : LERspGen: Attitude and Alignment correction of detector '10'  is 0.993464 !
lerspgen : LERspGen: Attitude and Al

```python -m pip install --index-url https://test.pypi.org/simple/ --no-deps hxmt-analysis-demo```

In [4]:
from hxmt_scripts.create_HE_specfile import create_HE_specfile

## 能谱分析